### SimpleMotor

In [ ]:
def compute(self, inputs, outputs):
    eta_m = self.options["efficiency"]
    weight_inc = self.options["weight_inc"]
    weight_base = self.options["weight_base"]
    cost_inc = self.options["cost_inc"]
    cost_base = self.options["cost_base"]
    outputs["shaft_power_out"] = inputs["throttle"] * inputs["elec_power_rating"] * eta_m
    outputs["heat_out"] = inputs["throttle"] * inputs["elec_power_rating"] * (1 - eta_m)
    outputs["elec_load"] = inputs["throttle"] * inputs["elec_power_rating"]
    outputs["component_cost"] = inputs["elec_power_rating"] * cost_inc + cost_base
    outputs["component_weight"] = inputs["elec_power_rating"] * weight_inc + weight_base
    outputs["component_sizing_margin"] = inputs["throttle"]

$$ P_{shaft,\ out} = Throttle\ []\ *\ Rated\ Electrical\ Power\ []\ *\ \eta_{motor}$$
$$ Heat_{out} = Throttle\ []\ *\ Rated\ Electrical\ Power\ []\ *\ (1\ -\ \eta_{motor})$$
$$ Load_{electrical} = Throttle\ []\ *\ Rated\ Electrical\ Power\ []$$
$$ Cost_{component} = (Rated\ Electrical\ Power\ []\ *\ Cost\ Inc\ Factor\ [])\ +\ Cost\ Base\ Factor\ []$$
$$ Weight_{component} = (Rated\ Electrical\ Power\ []\ *\ Weight\ Inc\ Factor\ [])\ +\ Weight\ Base\ Factor\ []$$


$$ J\ (Advance\ Ratio) = \frac{V_{a}\ [m/s]}{n\ [s-1]*D\ [m]}$$

### Propellor

#### Propellor Coefficients

In [ ]:
def compute(self, inputs, outputs):
    # print('Prop shaft power input: ' + str(inputs['shaft_power_in']))
    outputs["cp"] = (
        inputs["shaft_power_in"] / inputs["fltcond|rho"] / (inputs["rpm"] / 60) ** 3 / inputs["diameter"] ** 5
    )
    # print('cp: '+str(outputs['cp']))
    outputs["J"] = 60.0 * inputs["fltcond|Utrue"] / inputs["rpm"] / inputs["diameter"]
    # print('U:'+str(inputs['fltcond|Utrue']))
    # print('J: '+str(outputs['J']))
    outputs["prop_Vtip"] = inputs["rpm"] / 60 * np.pi * inputs["diameter"]

$$ cp\ (or)\ Power\ Coefficient =  \frac{P_{shaft,\ in}\ [kg\cdot m^{2}\cdot s^{-3}]}{rho\ [kg\cdot m^{-3}]\cdot (\frac{RPM}{60})^{3} [s^{-3}]\cdot Prop_{Dia}\ [m^5]} $$

$$ J\ (or)\ (Advance\ Ratio) = \frac{V_{a}\ [m/s]}{n\ [s-1]\cdot D\ [m]}$$

$$ Prop_{TipSpeed} = \frac{RPM}{60}[s^{-1}] \cdot \pi \cdot Prop_{Dia}\ [m]$$

Advance Ratio can be further divided to Static and Dynamic Advance Ratios. At Static regimes, vehicle is stationary or moving at low speeds. At Dynamic regime, vehicle is moving at significant forward speeds.

#### Thrust Calculations

In [ ]:
def compute(self, inputs, outputs):
        # for advance ratio j between 0.10 and 0.20, linearly interpolate the thrust coefficient from the two surrogate models
        jinterp_min = 0.10
        jinterp_max = 0.20
        j = inputs["J"]
        # print(inputs['eta_prop'])
        # Static Advance Ratio Index finding
        static_idx = np.where(j <= jinterp_min)
        # Dynamic Advance Ratio Index finding
        dynamic_idx = np.where(j >= jinterp_max)
        tmp = np.logical_and(j > jinterp_min, j < jinterp_max)
        interp_idx = np.where(tmp)

        cp = inputs["cp"]
        nn = self.options["num_nodes"]
        ct = np.zeros(nn)
        # ct1 and ct2 are two different modes of calculating thrust coefficient
        ct1 = inputs["ct_over_cp"] * cp
        ct2 = cp * inputs["eta_prop"] / j
        # For Static Advance Ratio
        # if j <= jinterp_min:
        ct[static_idx] = ct1[static_idx]

        # For some point between Static and Dynamic Advance Ratio
        # if j > jinterp_min and < jinterp_max:
        interv = np.ones(nn) * jinterp_max - np.ones(nn) * jinterp_min
        interp1 = (np.ones(nn) * jinterp_max - j) / interv
        interp2 = (j - np.ones(nn) * jinterp_min) / interv
        ct[interp_idx] = interp1[interp_idx] * ct1[interp_idx] + interp2[interp_idx] * ct2[interp_idx]

        # For Dynamic Advance Ratio
        # else if j >= jinterp_max
        ct[dynamic_idx] = ct2[dynamic_idx]

        outputs["thrust"] = ct * inputs["fltcond|rho"] * (inputs["rpm"] / 60.0) ** 2 * inputs["diameter"] ** 4

$$ ct1 = \frac{ct}{cp} \cdot cp $$
$$ ct2 = \frac{cp \cdot \eta_{Prop}}{j} $$
$$ Thrust = ct \cdot rho \cdot (\frac{RPM}{60})^{2} \cdot Prop_{Dia} $$

$$ cp = \frac{P_{shaft,\ in}}{\rho \cdot d \cdot (\frac{\Omega}{60})^3} $$

$$ cp = \frac{P_{shaft,\ in}\ [kg\cdot m^{2}\cdot s^{-3}]}{\rho\ [kg/m^3] \cdot d\ [m] \cdot (\frac{\Omega}{60})^3\ [s^{-3}]} $$